<a href="https://colab.research.google.com/github/hivewire/finetuning-phi-2-text-summarization/blob/main/Task_3_%E2%80%93_Fine_Tuning_Decoder_Only_LLM_(Phi_2)_for_Abstractive_Summarization_(XSum).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 3 – Fine-Tuning Decoder-Only LLM (Phi-2) for Abstractive Summarization (XSum)
**Tujuan Task**

Pada task ini dilakukan fine-tuning model decoder-only LLM (Phi-2) untuk menghasilkan ringkasan abstraktif singkat dari artikel berita menggunakan dataset XSum.

Karakteristik utama task:

**- Instruction-style prompting**

**- Causal Language Modeling**

**- Abstractive summarization (bukan ekstraktif)**

**- Generation control (max length, temperature, dll.)**

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)


# - Load Dataset
Tahap ini berfungsi untuk memuat dataset XSum dari file Parquet yang telah diekspor secara resmi. Pendekatan ini dipilih untuk menghindari penggunaan dataset script yang tidak lagi didukung oleh versi terbaru library HuggingFace datasets. Dataset yang dimuat berisi artikel berita dan ringkasan singkat yang sangat abstraktif, yang akan digunakan sebagai data pelatihan dan evaluasi model.

In [3]:
from datasets import load_dataset

data_files = {
    "train": "/content/sample_data/train-00000-of-00001.parquet",
    "validation": "/content/sample_data/validation-00000-of-00001.parquet",
    "test": "/content/sample_data/test-00000-of-00001.parquet",
}

ds = load_dataset("parquet", data_files=data_files)
print(ds)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})


# - Instruction-Style Prompt Formatting
Tahap ini melakukan transformasi data mentah menjadi format prompt berbasis instruksi. Setiap artikel berita digabungkan dengan instruksi eksplisit untuk melakukan peringkasan dan diikuti oleh ringkasan referensi. Format ini disesuaikan dengan karakteristik model Phi-2 yang merupakan decoder-only language model, sehingga tugas peringkasan dapat diperlakukan sebagai proses generasi teks berbasis instruksi.

In [4]:
def build_prompt(example):
    prompt = (
        "### Instruction:\n"
        "Summarize the following article in one concise sentence.\n\n"
        "### Article:\n"
        f"{example['document']}\n\n"
        "### Summary:\n"
        f"{example['summary']}"
    )
    return {"text": prompt}

ds = ds.map(
    build_prompt,
    remove_columns=ds["train"].column_names
)


# - Tokenizer Initialization
Tahap ini menginisialisasi tokenizer yang sesuai dengan model Phi-2. Tokenizer bertugas mengonversi teks menjadi representasi token numerik yang dapat diproses oleh model. Pada tahap ini juga ditentukan mekanisme padding agar seluruh input memiliki panjang yang seragam saat diproses dalam batch.

In [5]:
model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# - Tokenization
Tahap ini mengubah prompt berbasis teks menjadi urutan token numerik beserta attention mask. Proses tokenisasi memastikan bahwa setiap input memiliki panjang maksimum yang konsisten dan siap digunakan dalam proses pelatihan. Attention mask digunakan untuk memberi tahu model bagian mana dari input yang perlu diperhatikan.

In [6]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = ds.map(tokenize, batched=True)


Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

# - Label Assignment for Causal Language Modeling
Tahap ini menambahkan label pelatihan yang disamakan dengan input token. Hal ini diperlukan karena Phi-2 menggunakan pendekatan causal language modeling, di mana model dilatih untuk memprediksi token berikutnya berdasarkan token sebelumnya. Dengan adanya label ini, model dapat menghitung nilai loss dan melakukan proses pembaruan bobot selama pelatihan.

In [7]:
def add_labels(batch):
    batch["labels"] = batch["input_ids"].copy()
    return batch

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)


Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

# - Dataset Subsampling
Tahap ini membuat subset dari data pelatihan dengan mengambil sebagian kecil sampel dari dataset asli. Langkah ini dilakukan untuk menyesuaikan keterbatasan sumber daya komputasi, khususnya penggunaan GPU T4, serta untuk mempercepat proses fine-tuning tanpa mengubah alur metodologi secara keseluruhan.

In [8]:
train_subset = (
    tokenized_dataset["train"]
    .shuffle(seed=42)
    .select(range(200))
)


# - Model Initialization
Tahap ini memuat model Phi-2 yang telah dilatih sebelumnya dan menyiapkannya untuk proses fine-tuning. Model dikonfigurasikan agar menggunakan presisi FP16 untuk mengurangi penggunaan memori GPU dan meningkatkan efisiensi komputasi selama pelatihan.

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

for param in model.parameters():
    param.requires_grad = False

for param in model.lm_head.parameters():
    param.requires_grad = True

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
training_args = TrainingArguments(
    output_dir="./outputs/phi2-xsum",

    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    learning_rate=5e-4,

    logging_strategy="steps",
    logging_steps=1,

    save_strategy="no",

    fp16=False,
    dataloader_pin_memory=False,
    disable_tqdm=True,

    report_to="none"
)


# - Fine-Tuning with Trainer
Tahap ini menjalankan proses fine-tuning dengan menghubungkan model, dataset, dan parameter pelatihan melalui HuggingFace Trainer. Selama proses ini, model secara bertahap menyesuaikan bobotnya untuk menghasilkan ringkasan yang lebih ringkas dan abstraktif sesuai dengan instruksi. Keberhasilan pelatihan ditandai dengan penurunan nilai loss dan tersimpannya checkpoint model yang dapat digunakan untuk inferensi.

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
)

trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.


{'loss': 2.9219, 'grad_norm': 5.4921875, 'learning_rate': 0.0005, 'epoch': 0.005}
{'loss': 2.9353, 'grad_norm': 5.18359375, 'learning_rate': 0.0004975, 'epoch': 0.01}
{'loss': 14680.3369, 'grad_norm': 30.40625, 'learning_rate': 0.000495, 'epoch': 0.015}
{'loss': 18371.4766, 'grad_norm': 14.296875, 'learning_rate': 0.0004925, 'epoch': 0.02}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.00049, 'epoch': 0.025}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0004875, 'epoch': 0.03}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.00048499999999999997, 'epoch': 0.035}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0004825, 'epoch': 0.04}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.00048, 'epoch': 0.045}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0004775, 'epoch': 0.05}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.000475, 'epoch': 0.055}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0004725, 'epoch': 0.06}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate

TrainOutput(global_step=200, training_loss=165.28835376739502, metrics={'train_runtime': 20.5952, 'train_samples_per_second': 9.711, 'train_steps_per_second': 9.711, 'train_loss': 165.28835376739502, 'epoch': 1.0})

In [12]:
def summarize(article):
    prompt = (
        "### Instruction:\n"
        "Summarize the following article in one concise sentence.\n\n"
        "### Article:\n"
        f"{article}\n\n"
        "### Summary:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=40,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [13]:
trainer.save_model("./final-phi2-xsum")
tokenizer.save_pretrained("./final-phi2-xsum")


('./final-phi2-xsum/tokenizer_config.json',
 './final-phi2-xsum/special_tokens_map.json',
 './final-phi2-xsum/vocab.json',
 './final-phi2-xsum/merges.txt',
 './final-phi2-xsum/added_tokens.json',
 './final-phi2-xsum/tokenizer.json')